# Hashtag Grouping on Twitter

Input: 
    Hastag scrap result from cekmedsos_database.vw_ttidata

Output: 
    Mapping tables to group each hashtag entry with similarity
    

What we need to do....
1. Load the table entry from mySQL into python
2. Read the entry

Pre-requisite
pip install wheel
pip install pandas

### 1. Create Connection to mySQL

ref to this page:
    [How to Use Python with mySQL in Jupyter](https://medium.com/@tattwei46/how-to-use-python-with-mysql-79304bee8753)
    
first, we need to install mysql connector

In [1]:
import pymysql.cursors
import pandas as pd
import uuid
import time
import os

### Input as proces started
Record header and parameter information

 * Kesepakatan status di kolom screen_analisis_ai.status
 * 1 --> baru diinput
 * 2 --> lagi dikerjakan
 * 3 --> proses berhasil
 * 4 --> proses gagal
 *
 
 * Kesepakatan jenis analisa AI
 * 1 --> Analisa Cluster
 * 2 --> Analisa sentiment
 * 3 --> Analisa image clustering
 
 

### Marking this process as running
so that another process will not take this one


### Runnning Process


#### Starting process, 
Run query against RDBMS


In [5]:
#
# Query to get tweet data, apply analitics to this dataset
#

# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

s_query_string = 'select id, tweet, tweet_date_time from ret_tweet where '
# s_query_string = 'select id, tweet, tweet_date_time from vw_ret_tweet_clean where '

if (screen_name != ''):
    # print('use screen name')
    s_query_string = s_query_string + 'screen_name = "' + screen_name + '" and db_id = "' + str(database_keyword_id) + '"'
else:
    # print('no use')
    s_query_string = s_query_string + 'db_id = "' + database_keyword_id.replace('"','') + '"'
    
print(s_query_string)
df = pd.read_sql(s_query_string, con=connection)

# Close Connection
connection.close()

# see result
df.head()

select id, tweet, tweet_date_time from ret_tweet where db_id = "3507"


,id,tweet,tweet_date_time
0,1414011830247194625,Apa yang sedikit tetapi mencukupi lebih baik d...,2021-07-11 07:01:06
1,1414012129028411395,"Allah, tidak ada Tuhan melainkan Dia, Yang Hid...",2021-07-11 07:02:17
2,1414012255167913989,"Jika engkau berniat ingin bersedekah, atau mel...",2021-07-11 07:02:47
3,1414012548635004928,Dan sebutlah (nama) Tuhanmu sebanyak-banyaknya...,2021-07-11 07:03:57
4,1414012577642811393,Puasa itu tak menyakiti namun menyehatkan... m...,2021-07-11 07:04:04


### 2. Try to pre-process all the text

target-> tokenizing into another dataframe

In [6]:
import string
import re
import nltk

from nltk.tokenize import word_tokenize

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    # remove whitspace
    strOut = strOut.strip()
    # 
    strOut = re.sub('\s+',' ',strOut)
    
    return strOut
# end Text Preprocessing


# Apply to data frame
df['tweet'] = df['tweet'].apply(text_preproc)

df.head()

,id,tweet,tweet_date_time
0,1414011830247194625,apa yang sedikit tetapi mencukupi lebih baik d...,2021-07-11 07:01:06
1,1414012129028411395,allah tidak ada tuhan melainkan dia yang hidup...,2021-07-11 07:02:17
2,1414012255167913989,jika engkau berniat ingin bersedekah atau mela...,2021-07-11 07:02:47
3,1414012548635004928,dan sebutlah nama tuhanmu sebanyakbanyaknya se...,2021-07-11 07:03:57
4,1414012577642811393,puasa itu tak menyakiti namun menyehatkan mena...,2021-07-11 07:04:04


### Finish preprocessing, Tokenized
Next step, is to output to new column for tokenized sentence

In [20]:
from nltk.corpus import stopwords 

# Load stopwords
sw = stopwords.words('indonesian')

# apply tokenize
df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

# apply stopword removal
df['sw'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

df.head()

,id,tweet,tweet_date_time,tokenized_tweet,sw
0,1414011830247194625,apa yang sedikit tetapi mencukupi lebih baik d...,2021-07-11 07:01:06,"[apa, yang, sedikit, tetapi, mencukupi, lebih,...","{mencukupi, dawud, eyde, hrabu, ✅, sambutkejay..."
1,1414012129028411395,allah tidak ada tuhan melainkan dia yang hidup...,2021-07-11 07:02:17,"[allah, tidak, ada, tuhan, melainkan, dia, yan...","{allah, mengantuk, tidur, ⏱️, terusmenerus, sa..."
2,1414012255167913989,jika engkau berniat ingin bersedekah atau mela...,2021-07-11 07:02:47,"[jika, engkau, berniat, ingin, bersedekah, ata...","{atstsaury, menghalangi, amal, laksanakan, suf..."
3,1414012548635004928,dan sebutlah nama tuhanmu sebanyakbanyaknya se...,2021-07-11 07:03:57,"[dan, sebutlah, nama, tuhanmu, sebanyakbanyakn...","{petang, ☝️☝️, pagi, tuhanmu, ikz, bertasbihla..."
4,1414012577642811393,puasa itu tak menyakiti namun menyehatkan mena...,2021-07-11 07:04:04,"[puasa, itu, tak, menyakiti, namun, menyehatka...","{☝️☝️, menyakiti, selamat, lapar, puasa, menah..."


### Define function to calculate similarity
Function return similarity score

In [22]:
def calculate_similarity(X_set, Y_set):
# Program to measure the similarity between  
# two sentences using cosine similarity. 

    l1 =[];l2 =[]

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    
    return cosine

def largest_in_col(arr,nCol):
    # 
    # Find largest value of col nCol on 2D arr
    #
    
    # init value
    max_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] > max_val:
            max_val = arr[x][nCol]
            row_index = x
        
    return max_val,row_index

def smallest_in_col(arr,nCol):
    # 
    # Find smallest value of col nCol on 2D arr
    #
    
    # init value
    min_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] < min_val:
            min_val = arr[x][nCol]
            row_index = x
        
    return min_val,row_index


### Try to using function
using some of array cells, create n matrices

1. take one tweet, compare to all data set
2. flag 1 if similar
3. take next tweet, if similar from prev tweet, skip
4. if not similar, add counter, then proceed

In [23]:
import numpy as np
import array as arr
# import multitasking
from tqdm import tqdm

# set to max CPU Cores
# multitasking.set_max_threads(multitasking.config["CPU_CORES"] * 5)

st = similarity_treshold
cluster_no = 1
s_score = 0
s_score_current = 0
i_current_cluster = 0

#create zero element array
#col 0 => index base tweet
#col 1 => cluster number
#col 2 => similarity score
base_tweet = []

#proceed to compare to all tweet
cosine_matrix = np.zeros(( len(df), len(df) ), dtype=np.dtype('f4'))

# flag the mt
with tqdm(total=( (len(df)*len(df)/2))-(len(df)/2))  as pbar:
    for j in range(0, len(df)):
        tweet_to_compare = df['sw'][j]

        #check, is this second tweet?
        if(j == 0):
            #first tweet, add as cluster no #1
            base_tweet.append([j,1,1.0])
            i_current_cluster = base_tweet[0][1]

        elif(j == 1):
            #compare to prev tweet
            s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[0][0] ])

            if(s_score < st):
                #not similar
                base_tweet.append([j,2,1])
                i_current_cluster = base_tweet[j][1]

        else:
            #other else tweet
            for x in range(0,len(base_tweet)):
                #compare every element
                s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[x][0] ])
                base_tweet[x][2] = s_score

            if(largest_in_col(base_tweet,2)[0] < st):
                #no similar, add as one new cluster
                i_current_cluster = i_current_cluster + 1
                base_tweet.append([j,i_current_cluster,largest_in_col(base_tweet,2)[0]])
            else:
                #determine cluster# from biggest similarity
                i_current_cluster = base_tweet[(largest_in_col(base_tweet,2)[1])][1]

        #proceed to compare to all tweet
        for i in range(0,len(df)):
            # update progress
            if (j<i):
                pbar.update(1)
                s_score = calculate_similarity(tweet_to_compare, df['sw'][i])
                if (s_score >= st):
                    cosine_matrix[i,j] = i_current_cluster

        
pbar.close()
print(cosine_matrix)

100%|██████████| 28920/28920.0 [00:02<00:00, 11285.22it/s]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Writing result to file
Using NPZ format for efficiency, and try to load them after save


In [24]:
# save numpy array as npz file
from numpy import asarray
from numpy import savez_compressed

# save to npy file
# savez_compressed('./output/df_380.npz',df)
# savez_compressed('./output/data_380.npz', cosine_matrix)
# savez_compressed('./output/data_380_base.npz', base_tweet)

# print(len(cosine_matrix))
# print('Save data complete .... ')

from numpy import asarray
from numpy import savetxt

savetxt('data.csv', cosine_matrix, delimiter=',')

241
Save data complete .... 


### How to save to rdbms?
save the header data, meta data of the process
[ret_analysis_header]
- JobID
- user initiated
- Time Started
- Time End

Process Parameter
[ret_analysis_parameter]
- JobID
- Param Name
- Param Value
    - ST Value
    - DB ID
    - Screen Name
    - How Many Tweet analyzed

save detail cluster information
[ret_base_tweet]
- JobID
- Tweet Base# --> tweet ID
- Cluster#

save detail calculation result, structure
[ret_cluster_result]
- JobID
- TweetID
- Cluster

### Record cluster result
into table ret_cluster_result

In [18]:
from numpy import asarray
from numpy import savetxt

savetxt('data.csv', cosine_matrix, delimiter=',')

### Record finish time
update table ret_analysis_header

### Marking as finished the job
on this particular i_process_id

### Done Here ...
1. Create data structure to record below things
    Job Header
    - JobID
    - Similarity Treshold
    - Dataset Parameter
        - By User
        - By Keyword
    - Running time start
    - Running time end
    
    Dataset Result
        - base_tweet
        - cosine_matrix
        
2. Push the result data into RDBMS Server, in this case is mySQL
    - upload csv file to mySQL
    - import to database

In [15]:
# Wait 10 sec before release
time.sleep(10)